### In this hands on you will build a model that once trained on a peice of text data can generate its own sequnce of words in a similar fashion as in trained data.

- Follow the instructions provided for each cell and and code accordingly. 
- In order to run the cell press shift+enter.
- make sure you have run all the cells before submitting the hands on

### Run the below cell to import necessary packages

In [1]:
import numpy as np
import pandas as pd
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


### Read the text data from story.txt file and split the text into seperate tokens, assign thr array of tokens to variable *training_data*

### Expected output:
    array(['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general',
       'council', 'to']

In [6]:
### Start code here

from tokenize import tokenize
file_content = open("story.txt").read()
file_content
training_data = file_content.split()
###End code
training_data[:10]

['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general', 'council', 'to']

### Take the unique tokens in training_data nas sort them alphabetical order and assign the sorted list to variable words
### create dictionary ind_to_word to map index to word
### create another dictionary word_to_ind to reverse map word to their respective index.

In [21]:
####Start code here
words = list(sorted(set(training_data)))
ind_to_word = {}
for i,word in enumerate(words):
    ind_to_word[i]=word
word_to_ind = {}
for i,word in enumerate(words):
    word_to_ind[word]=i
###End code
print("words: ", words[:10], "\n")
print("index_to_words: ", list(ind_to_word.items())[:10], "\n")
print("word_to_index: ", list(word_to_ind.items())[:10], "\n")

words:  [',', '.', '?', 'a', 'about', 'ago', 'agree', 'all', 'always', 'an'] 

index_to_words:  [(0, ','), (1, '.'), (2, '?'), (3, 'a'), (4, 'about'), (5, 'ago'), (6, 'agree'), (7, 'all'), (8, 'always'), (9, 'an')] 

word_to_index:  [('said', 75), ('consider', 25), ('who', 106), ('?', 2), ('council', 28), ('neck', 58), ('was', 99), ('take', 83), ('ribbon', 73), ('impossible', 42)] 



### write a function to generate training dataset 
    - parameters: dataset: orginal dataset
                  look_back: the window size that tells the number of previous values in the series to look for to                   predict the next one.
    - returns: feature and target arrays
    
example: 
         for window size 1:
         dataset = [1,2,3,4]  
         feature = [[1],[2],[3]]    
         target = [2,3,4]  
         
         for window size 2:
         feature = [[1,2],[2,3]]  
         target = [3,4]  
### Expected output when you when you call generate_dataset on training_data and look_back = 10 :
input:  [[48, 5, 0, 85, 56, 37, 3, 35, 28, 92], [5, 0, 85, 56, 37, 3, 35, 28, 92, 25], [0, 85, 56, 37, 3, 35, 28, 92, 25, 102]]  
labels:  [25, 102, 53]


In [26]:
####Start code here
def generate_dataset(dataset, look_back=10):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        x=i+look_back
#         print(x)
        a = dataset[i:x]
        v=[]
        for k in a:
            v.append(word_to_ind[k])
        dataX.append(v)
        dataY.append(word_to_ind[dataset[x]])
    return np.array(dataX), np.array(dataY)
    
    
###End code

inputs, labels = generate_dataset(training_data, 10)
print("input: ", inputs[:3])
print("labels: ", labels[:3])

input:  [[ 48   5   0  85  56  37   3  35  28  92]
 [  5   0  85  56  37   3  35  28  92  25]
 [  0  85  56  37   3  35  28  92  25 102]]
labels:  [ 25 102  53]


### Next step is to  reshape the inputs and normalize them.
    - This step is coded for you
    - Run the below cell to prepare the data for training the model

In [27]:
look_back = 10
X_modified = np.reshape(inputs, (len(inputs), look_back, 1))

X_modified = X_modified / float(len(words))
Y_modified = np_utils.to_categorical(labels)

### Using keras Sequential() class create a model having two LSTM block and one fully connected  layer with activation softmax
### apply dropout with probability p = 0.2 between the layers of LSTM
### compile the model with categorical_crossentropy loss and  adam optimizer

### Expected output
<img src="lstm.png">

In [39]:
np.random.seed(51)
###Start code here

model = Sequential()
model.add(LSTM(400, return_sequences=True,input_shape=(X_modified.shape[1], X_modified.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))


###End code
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 10, 400)           643200    
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 400)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 400)               1281600   
_________________________________________________________________
dropout_9 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 112)               44912     
Total params: 1,969,712
Trainable params: 1,969,712
Non-trainable params: 0
_________________________________________________________________


### Run model.fit() on train data with features as X_modified and target as Y_modified for 50 epoches and batch_size = 10

In [41]:
###Start code here
train_logs = model.fit(X_modified, Y_modified, epochs=50,batch_size=10) 
###End code
with open("output.txt", "w+") as file:
    file.write("train score {0:.2f}\n".format(train_logs.history["loss"][-1]))

Epoch 1/50
193/193 [==============================] - 5s 24ms/step - loss: 4.6863
Epoch 2/50
193/193 [==============================] - 3s 17ms/step - loss: 4.5825
Epoch 3/50
193/193 [==============================] - 3s 17ms/step - loss: 4.5025
Epoch 4/50
193/193 [==============================] - 3s 16ms/step - loss: 4.4666
Epoch 5/50
193/193 [==============================] - 3s 16ms/step - loss: 4.4524
Epoch 6/50
193/193 [==============================] - 3s 16ms/step - loss: 4.4792
Epoch 7/50
193/193 [==============================] - 3s 17ms/step - loss: 4.4459
Epoch 8/50
193/193 [==============================] - 3s 17ms/step - loss: 4.4505
Epoch 9/50
193/193 [==============================] - 3s 17ms/step - loss: 4.4319
Epoch 10/50
193/193 [==============================] - 3s 17ms/step - loss: 4.4472
Epoch 11/50
193/193 [==============================] - 3s 16ms/step - loss: 4.4281
Epoch 12/50
193/193 [==============================] - 3s 18ms/step - loss: 4.4249
Epoch 13/50
1

### The below codes takes a random sequence of words and generates more sequnce using the model you trained above.

In [62]:
string_mapped = inputs[50].copy()
# print(string_mapped)
full_string = [ind_to_word[value] for value in string_mapped]
#generating characters
for i in range(100):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(words))
    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [ind_to_word[value] for value in string_mapped]
    full_string.append(ind_to_word[pred_index])
    string_mapped=list(string_mapped)
#     print(string_mapped,pred_index)
#     np.append(pred_index,string_mapped)
    string_mapped.append(pred_index)
    string_mapped=np.array(string_mapped)
    string_mapped = string_mapped[1:len(string_mapped)]

    
txt=""
for word in full_string:
    txt = txt+" "+word
print(txt)

 which he thought would meet the case . you will all agree , said he , that our chief danger consists what the sly and treacherous know approach , now agree , up another , and neck what venture old what some when neighbourhood . will agree met met met , said up that treacherous a is the an you old with she another the , their the proposal with we ? the you the will met met all , the proposal met with she was in take . this all agree agree , to up receive round , in neighbourhood . mice old , from a . spoke old
